In [ ]:
from unsloth import FastVisionModel
import torch
from PIL import Image

model, tokenizer = FastVisionModel.from_pretrained(
    "JPLabsAI/llava-latex_02",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

In [ ]:




FastVisionModel.for_inference(model)  # Enable for inference!

image = Image.open("integral_3.png")
instruction = """
Give me the LateX code for this equation.
"""

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": instruction},
        ],
    }
]
input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

output_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

# Decodificar
raw_result = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Quitar la parte de la "conversación" (user/assistant)
# Tomamos solo lo que está después de "assistant"
if "assistant" in raw_result:
    raw_result = raw_result.split("assistant", 1)[-1].strip()

# Quitar \begin{align*} y \end{align*}
clean_result = raw_result.replace(r"\begin{align*}", "").replace(r"\end{align*}", "").strip()

print("Resultado limpio:", clean_result)


In [ ]:


from sympy import integrate, Symbol
from sympy.parsing.latex import parse_latex

# LaTeX de la integral
latex_expr = clean_result

# Parseamos el LaTeX a Sympy
expr = parse_latex(latex_expr)

# Mostramos la expresión simbólica
print("Expresión en Sympy:", expr)

# Como es una Integral simbólica, la resolvemos
resultado = expr.doit()

print("Resultado:", resultado)
